## Extract data from pv and nwp

Extract big data from pv and nwp, by randomly choosing dates, and randomly choosing latitude and longitude. 

For those random points, choose, all steps and make a dataset. 

- select date, of day and then can try data.load to load data locally
- function in xarray, toDataframe(), can get dataframes, from xarray directly 
- and then concat for all days in a month. 

- randomly take some location and some times
- size of data: 1000 init times * 49 steps 
- 20, to 50 different sites. 

- can get 10 to 15 columns of ss_id and weather variables
- just keep ss_id, in resulting dataframe
- can get lat, long, capacity from metadata when need to feed to the model
- so our resulting dataframe will have ss_id and weather variables 10 to 15 columns


In [6]:
# select generation from pv_data , for each variable ss_id, and each date

# init time and daetime are not exact same from pv to nwp

# need to check nearest for datetime, what other options are available, 

# init time should be less than datetime,
# 
# 
# 
# training and testing, can keep 10 ids for trainign, 5 for inference
# 
# make your own repo on github and upload the code, for zak
# 
# use pv 30 min parquet file, and then select randomly . it is 9 gb
# pv.resample, data make it half an hour. get teh avergae of 6 values from 5 minutes each and use that vaue for 30 mins geenration. 
#  try to resample current one. 
#  

## Problems to fix
- nwp data has init_time, which is only a date and not hours, steps have hour for them, so nwp data is every hour. the init_time has either T00 or T12. Every date has T00 and T12, the program treats both dates separately. 

- pv has data for every date, every 5 minutes geenration captured. 

- so right now, when we are using nearest for datetime and init_time, it gets, one datetime from pv and keep it same for one date of init_time. 

- now when we selecting datetime from pv and then init_time from nwp, select first from pv, then find the time less than datetime from nwp. from nwp, can be less than, but can't be more than nwp time. 

- now problemis how to get time from nwp, need to consider both init_time and steps. 

- let's take for an hour from pv. 

## confusion
we are only selecting date from pv data sets, because for that date we are taking all steps, at every hour from nwp. if we select hour too, we need to set step too at that fixed time. so it mean, when i select a date from pv data, i need to get all hours data from it. 


> For one time in datetime, if geenration is 0, it will be same for all steps in nwp data. so no use of selecting only date, we need to select both date and hours, similar in both datsets.

## solution

So actually init_time is the time at which the fprecast is made, and steps are the forecast at subsequent steps, each hour. So, no steps and init_time combined are not date and time. only init_time is the date that i need to match with datetime from pv.  And then for time, 

now, let's see init_time, it has only 2 hours, T00 and T12, so we get the pv date and time, and from that we get the nearest init_time, it can either be for that date, T00 or T12. So for one pv time, just take init_time closest to it, and all steps. if T00 or T12 is missing, it can go to the last previuous date found.  

In [1]:
import xarray as xr
import ocf_blosc2
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
# import matplotlib.pyplot as plt
# import plotly.express as px
import numpy as np


In [2]:
nwp_data = xr.open_dataset("../../../mnt/disks/gcp_data/nwp/ecmwf/UK_v2.zarr")
nwp_data

<xarray.Dataset> Size: 1TB
Dimensions:    (variable: 14, init_time: 5049, step: 49, latitude: 241,
                longitude: 301)
Coordinates:
  * init_time  (init_time) datetime64[ns] 40kB 2017-01-01 ... 2023-12-31T12:0...
  * latitude   (latitude) float64 2kB 60.0 59.95 59.9 59.85 ... 48.1 48.05 48.0
  * longitude  (longitude) float64 2kB -12.0 -11.95 -11.9 ... 2.9 2.95 3.0
  * step       (step) timedelta64[ns] 392B 00:00:00 01:00:00 ... 2 days 00:00:00
  * variable   (variable) <U5 280B 't2m' 'dswrf' 'dlwrf' ... 'v10' 'u100' 'v100'
Data variables:
    ECMWF_UK   (variable, init_time, step, latitude, longitude) float32 1TB ...

In [3]:
pv_data = xr.open_dataset("data_files/pv.netcdf", engine='h5netcdf')
pv_data

<xarray.Dataset> Size: 2GB
Dimensions:   (datetime: 387254)
Coordinates:
  * datetime  (datetime) datetime64[ns] 3MB 2018-01-01T06:05:00 ... 2021-10-2...
Data variables: (12/1311)
    10003     (datetime) float32 2MB ...
    10004     (datetime) float32 2MB ...
    10005     (datetime) float32 2MB ...
    10048     (datetime) float32 2MB ...
    10049     (datetime) float32 2MB ...
    10063     (datetime) float32 2MB ...
    ...        ...
    9870      (datetime) float32 2MB ...
    9871      (datetime) float32 2MB ...
    9902      (datetime) float32 2MB ...
    9903      (datetime) float32 2MB ...
    9960      (datetime) float32 2MB ...
    9989      (datetime) float32 2MB ...

In [4]:

meta_data = pd.read_csv("data_files/metadata.csv")
meta_data.head()

,ss_id,latitude_rounded,longitude_rounded,llsoacd,orientation,tilt,kwp,operational_at
0,2405,53.53,-1.63,E01007430,180.0,35.0,3.36,2010-11-18
1,2406,54.88,-1.38,E01008780,315.0,30.0,1.89,2010-12-03
2,2407,54.88,-1.38,E01008780,225.0,30.0,1.89,2010-12-03
3,2408,54.88,-1.38,E01008780,225.0,30.0,1.89,2010-12-03
4,2409,54.88,-1.38,E01008780,225.0,30.0,1.89,2010-12-03


#### first step

- need to do one thing, need to take every half an hour data
- because in nwp data is every half an hour

- randomly choose pv sites, did it manually
- also need to select date time from pv_site
- randomly select 3 values from pv_site for each ss_id
- ss_id can be a key of dictionary, and date, latitude, longitude can be values
- date is not going to be one value liek latitude and long, it is going to be mulitple values 
- so i can maintain two dictionaries too, one with lat, long, and one with date time values
- i should make one dataframe with lat , long , is and dates. 
- and then go to nwp and get the values of variables in it.

- need to get 3 things from pv_data, ss_id, datetime and generation. generation is amount of power generated at that time. 

# TFT
need continous 48 hours data for tft. so select a date and select all steps for it from pv data first and then do the same for nwp dataset.

In [5]:
pv_data
# pv_sites_id = [10064, 10168, 10528, 12091, 17073, 26846, 2812, 7050, 9902, 3324]
# print(pv_sites_id)

hourly_pv_data = pv_data.sel(datetime = pv_data['datetime'].dt.minute == 0)

# hourly_pv_data = pv_data.sel(datetime = pv_data['datetime'].dt.minute == 0 | (pv_data['datetime'].dt.minute == 30))

data_vars = list(pv_data.data_vars)
pv_sites_id = np.random.choice(data_vars, 25, replace=False)
print(pv_sites_id)

data_dict = {'ss_id': [], 'pv_datetime': [], 'generation' : []}

for ss_id in pv_sites_id:
    random_dates = np.random.choice(hourly_pv_data['datetime'].values, 700, replace=False)
    power_values = hourly_pv_data[ss_id].sel(datetime=random_dates).values

    for date, power in zip(random_dates, power_values):
        data_dict['ss_id'].append(int(ss_id))
        data_dict['pv_datetime'].append(date)
        data_dict['generation'].append(power)




    # data_dict['ss_id'].extend([int(ss_id)] * len(random_dates))
    # data_dict['date'].extend(random_dates)
    # data_dict['generation'].extend(power_values)
    
   

    # taking too much time, this apporach and loop
    # for date in random_dates:
    #     data_dict['ss_id'].append(int(ss_id))
    #     data_dict['date'].append(date)

    #     power = pv_data.sel(datetime=date)[ss_id].values
    #     data_dict['generation'].append(power)

pv_df = pd.DataFrame(data_dict)

pv_df = pv_df.dropna(subset={'generation'})

pv_df


['26907' '26803' '26835' '7351' '26878' '7487' '7448' '10531' '26891'
 '26897' '6038' '3026' '26782' '6127' '3266' '6975' '7479' '26868' '11712'
 '12886' '27005' '26900' '7835' '6949' '26813']


,ss_id,pv_datetime,generation
8,26907,2018-12-23 01:00:00,0.000000
12,26907,2018-06-21 20:00:00,8.082132
14,26907,2019-09-18 02:00:00,0.000000
15,26907,2019-07-10 09:00:00,821.687988
16,26907,2020-02-15 05:00:00,0.000000
...,...,...,...
17495,26813,2018-11-20 00:00:00,0.000000
17496,26813,2019-03-04 05:00:00,0.000000
17497,26813,2019-04-02 23:00:00,0.000000
17498,26813,2020-01-19 15:00:00,1340.843994


In [9]:
pv_df.to_csv('result_data/pv_generaton.csv')


#### second step to find random dates and latitude, longitude for these sites. 
- that can be done from metadata
- one approach is to make a new dictioanry with key as ss_id an dpngitude and latitiude as values elements. 
- or make a dictionary and then make a dataframe out of it.


In [10]:
pv_sites_id = [int(id) for id in pv_sites_id]  
# pv_sites_id

In [11]:
pv_site_dict = {'ss_id':[], "lat":[], "long": []}

for id in pv_sites_id:
    row = meta_data[meta_data['ss_id'] == id]
    if not row.empty:
        pv_site_dict['ss_id'].append(id)
        pv_site_dict['lat'].append(row['latitude_rounded'].values[0])
        pv_site_dict['long'].append(row['longitude_rounded'].values[0])
    else:
        print('row empty')
        


print(pv_site_dict)
meta_site_df = pd.DataFrame.from_dict(pv_site_dict)
meta_site_df

{'ss_id': [6618, 6871, 7766, 2867, 6506, 27018, 26811, 9350, 26932, 3536, 7248, 26788, 11441, 9101, 10840, 7339, 9989, 3249, 26864, 10528, 4029, 12846, 7390, 6932, 26803], 'lat': [50.88, 51.56, 53.93, 55.81, 55.83, 51.33, 53.68, 54.54, 51.85, 51.95, 55.31, 51.33, 56.09, 53.43, 57.63, 55.34, 54.9, 55.64, 51.26, 54.54, 51.3, 51.66, 55.94, 53.21, 52.33], 'long': [-1.03, 0.17, -1.8, -4.14, -4.49, -2.17, -2.45, -1.93, -2.21, -0.29, -4.4, -2.87, -3.58, -2.77, -3.31, -1.57, -1.55, -4.28, -2.2, -1.3, 0.45, -3.31, -4.77, -0.58, -0.07]}


,ss_id,lat,long
0,6618,50.88,-1.03
1,6871,51.56,0.17
2,7766,53.93,-1.80
3,2867,55.81,-4.14
4,6506,55.83,-4.49
5,27018,51.33,-2.17
6,26811,53.68,-2.45
7,9350,54.54,-1.93
8,26932,51.85,-2.21
9,3536,51.95,-0.29


In [12]:
# combine both dates, with lat and long of pv site

combined_df = pd.merge(pv_df, meta_site_df, on='ss_id', how='inner')
combined_df


,ss_id,pv_datetime,generation,lat,long
0,6618,2021-03-13 03:00:00,0.000000,50.88,-1.03
1,6618,2018-10-14 19:00:00,0.000000,50.88,-1.03
2,6618,2018-08-13 14:00:00,2992.691895,50.88,-1.03
3,6618,2019-12-01 21:00:00,0.000000,50.88,-1.03
4,6618,2019-04-26 09:00:00,1361.556030,50.88,-1.03
...,...,...,...,...,...
12148,26803,2019-01-23 00:00:00,0.000000,52.33,-0.07
12149,26803,2018-04-29 19:00:00,0.000000,52.33,-0.07
12150,26803,2019-03-07 19:00:00,0.000000,52.33,-0.07
12151,26803,2018-06-08 10:00:00,1345.176025,52.33,-0.07


#### Third step
- now we have a dictionay of random pv_sites and their latitude and longitude.
- next step is to go to nwp, sleect 10 variables and for those 10 variables select values
- go through the dictionary and select all steps and 10 variables. 

In [15]:
print(combined_df.shape)
# ['t2m', 'dswrf', 'dlwrf', 'sde', 'sr', 'tcc', 'hcc', 'lcc', 'mcc',
#    'duvrs', 'u10', 'v10', 'u100', 'v100']

# List to collect DataFrames
dataframes = []

(12153, 5)


- 10 sites, 3 datetimes, 30 rows
- 30* 49 steps = 1470

In [22]:
for index, row in combined_df.iterrows():
    ss_id = row['ss_id']
    data_sel = nwp_data.sel(latitude=row['lat'], method="nearest") \
                       .sel(longitude=row['long'], method="nearest") \
                       .sel(init_time=row['pv_datetime'], method="pad") #backfill

    data_df = data_sel.to_dataframe().reset_index()
    pivot_df = data_df.pivot_table(index=['init_time', 'step'], columns='variable', values='ECMWF_UK').reset_index()

    pivot_df['ss_id'] = ss_id
    pivot_df['pv_datetime'] = row['pv_datetime']
    pivot_df['generation'] = row['generation']

    dataframes.append(pivot_df)

In [30]:
final_df = pd.concat(dataframes, ignore_index=True)
print(final_df.shape)

(596967, 19)


In [31]:

desired_order = ['ss_id', 'init_time', 'step', 'generation', 'dlwrf', 'dswrf', 'duvrs', 'hcc', 'lcc', 'mcc', 'sde', 'sr', 't2m', 'tcc', 'u10', 'u100', 'v10', 'v100', 'pv_datetime']
final_df = final_df[desired_order]


- 25 pv sites
- 500 dates for each
- 25 * 500 = 12500 rows
- 12500 * 49 steps = 612500 rows
- took 15 minutes for 612500 25 sites, 500 dates each
- each site has 24500 rows, 1* 500 * 49 = 24,500

In [32]:
final_df.head()

variable,ss_id,init_time,step,generation,dlwrf,dswrf,duvrs,hcc,lcc,mcc,sde,sr,t2m,tcc,u10,u100,v10,v100,pv_datetime
0,6618,2021-03-13 12:00:00,0 days 00:00:00,0.0,0.000000e+00,0.0,0.00000,0.120270,0.213715,0.503693,0.0,0.0,281.627441,0.593079,9.075681,12.959560,0.391643,0.402195,2021-03-13 03:00:00
1,6618,2021-03-13 12:00:00,0 days 01:00:00,0.0,9.669918e+05,1986408.5,195101.65625,0.283463,0.179688,0.769409,0.0,2424832.0,282.107422,0.789007,8.568085,12.093464,0.462411,0.442393,2021-03-13 03:00:00
2,6618,2021-03-13 12:00:00,0 days 02:00:00,0.0,2.009292e+06,3610686.0,380125.62500,0.504700,0.249512,0.239410,0.0,4213760.0,281.720947,0.627586,8.403449,11.935187,-0.338459,-0.619168,2021-03-13 03:00:00
3,6618,2021-03-13 12:00:00,0 days 03:00:00,0.0,3.012717e+06,4813532.0,512753.06250,0.761642,0.190979,0.946503,0.0,5491302.5,280.990234,0.959890,8.507233,12.369464,-0.521250,-0.932540,2021-03-13 03:00:00
4,6618,2021-03-13 12:00:00,0 days 04:00:00,0.0,4.088206e+06,5552847.0,603310.50000,0.430542,0.151154,0.627014,0.0,5980931.5,280.312988,0.686188,7.880438,11.654131,-0.599317,-1.071305,2021-03-13 03:00:00


In [33]:
row = final_df[final_df['ss_id'] == 6618]
print(row.shape)
row.head()

(35182, 19)


variable,ss_id,init_time,step,generation,dlwrf,dswrf,duvrs,hcc,lcc,mcc,sde,sr,t2m,tcc,u10,u100,v10,v100,pv_datetime
0,6618,2021-03-13 12:00:00,0 days 00:00:00,0.0,0.000000e+00,0.0,0.00000,0.120270,0.213715,0.503693,0.0,0.0,281.627441,0.593079,9.075681,12.959560,0.391643,0.402195,2021-03-13 03:00:00
1,6618,2021-03-13 12:00:00,0 days 01:00:00,0.0,9.669918e+05,1986408.5,195101.65625,0.283463,0.179688,0.769409,0.0,2424832.0,282.107422,0.789007,8.568085,12.093464,0.462411,0.442393,2021-03-13 03:00:00
2,6618,2021-03-13 12:00:00,0 days 02:00:00,0.0,2.009292e+06,3610686.0,380125.62500,0.504700,0.249512,0.239410,0.0,4213760.0,281.720947,0.627586,8.403449,11.935187,-0.338459,-0.619168,2021-03-13 03:00:00
3,6618,2021-03-13 12:00:00,0 days 03:00:00,0.0,3.012717e+06,4813532.0,512753.06250,0.761642,0.190979,0.946503,0.0,5491302.5,280.990234,0.959890,8.507233,12.369464,-0.521250,-0.932540,2021-03-13 03:00:00
4,6618,2021-03-13 12:00:00,0 days 04:00:00,0.0,4.088206e+06,5552847.0,603310.50000,0.430542,0.151154,0.627014,0.0,5980931.5,280.312988,0.686188,7.880438,11.654131,-0.599317,-1.071305,2021-03-13 03:00:00


In [34]:
final_df.shape

(596967, 19)

In [35]:
final_df.to_csv("result_data/final_data_half_hour.csv")

In [121]:
final_df.head()

variable,ss_id,init_time,step,dlwrf,dswrf,duvrs,hcc,lcc,mcc,sde,sr,t2m,tcc,u10,u100,v10,v100,datetime
0,7547,2019-09-22 12:00:00,0 days 00:00:00,0.0,0.000000e+00,0.00000,0.982452,0.013000,0.933258,0.0,0.0,291.135742,0.992920,-2.428142,-3.754718,-1.049166,0.154392,2019-09-22 09:15:00
1,7547,2019-09-22 12:00:00,0 days 01:00:00,1353008.0,5.504762e+05,71664.53125,0.933594,0.001801,0.986847,0.0,250496.0,290.958496,0.998718,-3.026777,-4.564105,-0.490215,1.016706,2019-09-22 09:15:00
2,7547,2019-09-22 12:00:00,0 days 02:00:00,2712773.0,1.003392e+06,141429.59375,0.362244,0.000092,0.777679,0.0,334080.0,290.792236,0.841644,-3.483191,-5.351845,-0.659222,0.992348,2019-09-22 09:15:00
3,7547,2019-09-22 12:00:00,0 days 03:00:00,4054373.0,1.545526e+06,218643.50000,0.110321,0.009705,0.680908,0.0,677632.0,291.014404,0.703888,-3.351099,-5.051410,0.102624,2.067908,2019-09-22 09:15:00
4,7547,2019-09-22 12:00:00,0 days 04:00:00,5361400.0,2.148786e+06,286584.31250,0.503754,0.010254,0.870911,0.0,1538816.0,291.002930,0.923370,-2.785363,-3.987979,0.761361,3.078596,2019-09-22 09:15:00


In [3]:
df = pd.read_csv("result_data/final_data.csv")

In [36]:
df_half = pd.read_csv("result_data/final_data_half_hour.csv")

In [37]:
df_half

,Unnamed: 0,ss_id,init_time,step,generation,dlwrf,dswrf,duvrs,hcc,lcc,mcc,sde,sr,t2m,tcc,u10,u100,v10,v100,pv_datetime
0,0,6618,2021-03-13 12:00:00,0 days 00:00:00,0.0,0.0,0.0,0.00,0.120270,0.213715,0.503693,0.0,0.0,281.62744,0.593079,9.075681,12.959560,0.391643,0.402195,2021-03-13 03:00:00
1,1,6618,2021-03-13 12:00:00,0 days 01:00:00,0.0,966991.8,1986408.5,195101.66,0.283463,0.179688,0.769409,0.0,2424832.0,282.10742,0.789007,8.568085,12.093464,0.462411,0.442393,2021-03-13 03:00:00
2,2,6618,2021-03-13 12:00:00,0 days 02:00:00,0.0,2009292.0,3610686.0,380125.62,0.504700,0.249512,0.239410,0.0,4213760.0,281.72095,0.627586,8.403449,11.935187,-0.338459,-0.619168,2021-03-13 03:00:00
3,3,6618,2021-03-13 12:00:00,0 days 03:00:00,0.0,3012717.0,4813532.0,512753.06,0.761642,0.190979,0.946503,0.0,5491302.5,280.99023,0.959890,8.507233,12.369464,-0.521250,-0.932540,2021-03-13 03:00:00
4,4,6618,2021-03-13 12:00:00,0 days 04:00:00,0.0,4088206.0,5552847.0,603310.50,0.430542,0.151154,0.627014,0.0,5980931.5,280.31300,0.686188,7.880438,11.654131,-0.599317,-1.071305,2021-03-13 03:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596962,596962,26803,2019-05-11 00:00:00,1 days 20:00:00,0.0,47339776.0,42029410.0,4506912.00,0.029846,0.052307,0.308228,0.0,42848430.0,283.20288,0.332123,-2.831519,-5.723359,0.691452,1.790067,2019-05-11 00:00:00
596963,596963,26803,2019-05-11 00:00:00,1 days 21:00:00,0.0,48342064.0,42029440.0,4506861.00,0.093094,0.000946,0.050903,0.0,42848560.0,281.50342,0.139221,-1.973587,-4.020501,1.634062,4.033380,2019-05-11 00:00:00
596964,596964,26803,2019-05-11 00:00:00,1 days 22:00:00,0.0,49318770.0,42029440.0,4506861.00,0.222290,0.000031,0.014374,0.0,42848560.0,280.35327,0.233368,-1.355677,-2.814991,2.076920,4.882574,2019-05-11 00:00:00
596965,596965,26803,2019-05-11 00:00:00,1 days 23:00:00,0.0,50293550.0,42029440.0,4506861.00,0.000000,0.000000,0.006531,0.0,42848560.0,279.54712,0.006531,-0.687275,-1.736615,1.930042,4.882493,2019-05-11 00:00:00
